#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)  
3) измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [316]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
from pymystem3 import Mystem
mystem = Mystem()
import pandas as pd
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from nltk.tokenize import word_tokenize, wordpunct_tokenize
import matplotlib.pyplot as plt
import re

In [121]:
wiki = open('wiki_data.txt', encoding="utf-8").read().split('\n')


In [355]:
##попробавала так, но не работало 
def preprocess(text):
    tokens = [morph.parse(token)[0][2] for token in word_tokenize(text)]
    for token in tokens:
        if re.search('а-я' ,token):
            token.strip(punctuation)
        else: 
            tokens.remove(token)
    return tokens



In [427]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [428]:

'''vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))'''

In [438]:
cv = CountVectorizer(max_features=10000)
svd = TruncatedSVD(200)
X = cv.fit_transform(wiki)


id2word = cv.get_feature_names()
word2id = {word:i for i,word in enumerate(id2word)}

In [439]:
vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))

In [441]:
sentences = []

for text in wiki:
    tokens = preprocess(text)

    ids = [word2id[token] for token in tokens if token in word2id]

    sentences.append(ids)
    print(sentences)

    


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [442]:
# СВОW
X = []
y = []

window = 6
for sent in sentences[:10000]:
    for i in range(len(sent)-1):
        word = sent[i]
        context = sent[max(0, i-window):i] + sent[i+1:i+window]
        
        X.append(context)
        y.append(word)


In [443]:
# оригинальные контексты
np.array(X[:10])

C:\Users\123ha\AppData\Local\Temp/ipykernel_4080/3626759454.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X[:10])


array([list([4893, 7700, 6375, 2210, 7116]),
       list([4935, 7700, 6375, 2210, 7116, 4895]),
       list([4935, 4893, 6375, 2210, 7116, 4895, 5023]),
       list([4935, 4893, 7700, 2210, 7116, 4895, 5023, 1500]),
       list([4935, 4893, 7700, 6375, 7116, 4895, 5023, 1500, 8152]),
       list([4935, 4893, 7700, 6375, 2210, 4895, 5023, 1500, 8152, 7559]),
       list([4935, 4893, 7700, 6375, 2210, 7116, 5023, 1500, 8152, 7559, 7154]),
       list([4893, 7700, 6375, 2210, 7116, 4895, 1500, 8152, 7559, 7154, 3485]),
       list([7700, 6375, 2210, 7116, 4895, 5023, 8152, 7559, 7154, 3485, 9928]),
       list([6375, 2210, 7116, 4895, 5023, 1500, 7559, 7154, 3485, 9928, 5450])],
      dtype=object)

In [444]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=10, padding='post')
y = np.array(y)

In [445]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [446]:
modelCB = tf.keras.Sequential()

# 1 отличие - то что на вход подается не 1 слово а целое окно контекста
# но на уровне embedding слоя нужно изменить только input_length
# она будет равна числу до которого мы делали паддинг
modelCB.add(tf.keras.layers.Embedding(input_dim=len(word2id), 
                                    input_length=10, 
                                    output_dim=100))

# 2 отличие - нужно усреднить контекст в 1 вектор
modelCB.add(tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1)))

modelCB.add(tf.keras.layers.Dense(len(word2id), activation='softmax'))
modelCB.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy',
              
              )

C:\Users\123ha\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [447]:
print(modelCB.summary())


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, 10, 100)           1000000   
                                                                 
 lambda_23 (Lambda)          (None, 100)               0         
                                                                 
 dense_23 (Dense)            (None, 10000)             1010000   
                                                                 
Total params: 2,010,000
Trainable params: 2,010,000
Non-trainable params: 0
_________________________________________________________________
None


In [448]:

modelCB.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=5)

Epoch 1/5
144/144 [==============================] - 21s 141ms/step - loss: 8.4963 - val_loss: 8.0222
Epoch 2/5
144/144 [==============================] - 20s 140ms/step - loss: 7.7843 - val_loss: 7.7436
Epoch 3/5
144/144 [==============================] - 21s 145ms/step - loss: 7.4058 - val_loss: 7.4827
Epoch 4/5
144/144 [==============================] - 23s 161ms/step - loss: 7.0547 - val_loss: 7.2840
Epoch 5/5
144/144 [==============================] - 24s 167ms/step - loss: 6.7216 - val_loss: 7.1276


In [449]:
embeddings_1 = modelCB.layers[0].get_weights()[0]
embeddings_2 = modelCB.layers[2].get_weights()[0].T
embeddingsCB = np.mean([embeddings_1, embeddings_2], axis=0)

In [450]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [451]:

most_similar('улица', embeddingsCB)

['улица',
 'названа',
 'челябинской',
 'районе',
 'житомирская',
 'ныне',
 'называлась',
 'сельского',
 'деревня',
 'пермской']

In [452]:
most_similar('знать', embeddingsCB)

['знать',
 'задачу',
 'факт',
 'хочет',
 'нельзя',
 'всегда',
 'настолько',
 'долго',
 'мы',
 'вселенной']

In [453]:
#SKIP GRAM
X = []
y = []

window = 12
for sent in sentences[:1000]:
    for i in range(len(sent)-1):
        word = sent[i] 
        context = sent[max(0, i-window):i] + sent[i+1:i+window] 

        for context_word in context:
            X.append(word)
            y.append(context_word)

In [454]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [455]:
# инициализируем модель

model = tf.keras.Sequential()


model.add(tf.keras.layers.Embedding(input_dim=len(word2id), 
                                    input_length=1, 
                                    output_dim=100)) # также указываем желаемый размер эмбединга

model.add(tf.keras.layers.Flatten()) 

model.add(tf.keras.layers.Dense(len(word2id), activation='softmax'))



model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), 
              loss='sparse_categorical_crossentropy',
              
              )

C:\Users\123ha\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [457]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000, 
         epochs=4)

Epoch 1/4
3190/3190 [==============================] - 694s 218ms/step - loss: 7.6194 - val_loss: 7.5370
Epoch 2/4
 941/3190 [=======>......................] - ETA: 8:37 - loss: 7.4472

KeyboardInterrupt: 

In [ ]:
embeddings_1 = model.layers[0].get_weights()[0]
embeddings_2 = model.layers[2].get_weights()[0].T

In [ ]:
embeddingsSG = np.mean([embeddings_1, embeddings_2], axis=0)

In [ ]:
most_similar('улица', embeddingsSG)

In [ ]:
most_similar('знать', embeddingsSG)

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [ ]:
import gensim


In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [ ]:
texts = [preprocess(text) for text in wiki]


In [ ]:

w2v = gensim.models.Word2Vec(texts, 
                             vector_size=50, 
                             min_count=40, 
                             max_vocab_size=1000,
                             window=7,
                             epochs=5,
                            sg=1)

In [ ]:
ft = gensim.models.FastText(texts, min_n=3, max_n=7)

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [ ]:
data = pd.read_csv('labeled.csv')
data['norm_text'] = data.comment.apply(preprocess)

In [ ]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)



In [ ]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}

In [ ]:
X = []

for tokens in data['norm_text']:
    ids = [word2id[token] for token in tokens if token in word2id]
    X.append(ids)

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=100)
y = data.toxic.values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [ ]:
inputs = tf.keras.layers.Input(shape=(100,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs, )
# embedding слой возвращает последовательность векторов
# а нам нужно классифицировать сразу весь текст
# стандартный подход в этом случае - усреднить единичные вектора в 1 вектор текста
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)
# к усредненному вектору мы уже применяем полносвязный слой, который вернет вероятность токсичности
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

In [ ]:
weights = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    # вектор паддинга оставим нулевым
    if word == 'PAD':
        continue

    try:
        weights[i] = ft.wv[word]
    
    
    except KeyError:
        # для слов, которых нет в модели тоже возьмем какой-то  рандомный вектор
        continue
        weights[i] = ft.wv['опрагпллирао']

In [ ]:
inputs = tf.keras.layers.Input(shape=(100,))

# передаем матрицу в эмбединг слой
# указываем параметр trainable=False, чтобы вектора не обучались
# Пояснение: кажется, что обучение векторов отключать не стоит, так как модель должна взять готовые 
# эмбединги и подогнать их к нашей задаче. На практитке это однако не работает - эмбединги просто 
# обучатся заново
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

In [ ]:
weights = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    # вектор паддинга оставим нулевым
    if word == 'PAD':
        continue

    try:
        weights[i] = ft.wv[word]
    
    
    except KeyError:
        # для слов, которых нет в модели тоже возьмем какой-то  рандомный вектор
        continue
        weights[i] = ft.wv['опрагпллирао']

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=30)

In [ ]:
print(model.history.history.keys())
# summarize history for accuracy
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
vocab = Counter()

for text in data['norm_text']:
    vocab.update(text)
    
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)



In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=50)
y = data.toxic.values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [ ]:
inputs = tf.keras.layers.Input(shape=(50,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs, )
# embedding слой возвращает последовательность векторов
# а нам нужно классифицировать сразу весь текст
# стандартный подход в этом случае - усреднить единичные вектора в 1 вектор текста
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)
# к усредненному вектору мы уже применяем полносвязный слой, который вернет вероятность токсичности
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=10)

In [ ]:
weights = np.zeros((len(word2id), ))

for word, i in word2id.items():
   # if i>99:
   # #    break
    # вектор паддинга оставим нулевым
    if word == 'PAD':
        continue

    try:
        weights[i] = modelCB.layers[0].get_weights()[0][i][0]
    
    
    except KeyError:
        # для слов, которых нет в модели тоже возьмем какой-то  рандомный вектор
        continue
        weights[i] = modelCB.wv['опрагпллирао']

In [ ]:
inputs = tf.keras.layers.Input(shape=(50,))
# передаем матрицу в эмбединг слой
# указываем параметр trainable=False, чтобы вектора не обучались
# Пояснение: кажется, что обучение векторов отключать не стоит, так как модель должна взять готовые 
# эмбединги и подогнать их к нашей задаче. На практитке это однако не работает - эмбединги просто 
# обучатся заново
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=50, 
                                       trainable=False,
                                       weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=32,
         epochs=30)

In [ ]:
print(model.history.history.keys())
# summarize history for accuracy
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()